# Tutorial 54: Tables

This example demonstrates how view, create, delete and edit measured variable tables (Sollwerttabellen) with pandas dataframes as python representation.

# SIR 3S Installation

In [1]:
SIR3S_SIRGRAF_DIR = r"C:\3S\SIR 3S\SirGraf-90-15-00-22_Quebec-Upd2" #change to local path

# Imports

In [2]:
from sir3stoolkit.core import wrapper

In [3]:
from sir3stoolkit.mantle import mantle

The wrapper package has to be initialized with reference to a SIR 3S (SirGraf) installation.

In [4]:
wrapper.Initialize_Toolkit(SIR3S_SIRGRAF_DIR)

## Additional imports

In [5]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

...

# Initialization

In [6]:
s3s = mantle.SIR3S_Model_Mantle()

Initialization complete


In [7]:
from sir3stoolkit.mantle import dataframes
s3s = dataframes.SIR3S_Model_Dataframes()

Initialization complete


# Open Model

In [20]:
dbFilePath=r"Toolkit_Tutorial54_Model.db3"

In [21]:
s3s.OpenModel(dbName=dbFilePath,
              providerType=s3s.ProviderTypes.SQLite,
              Mid="M-1-0-1",
              saveCurrentlyOpenModel=False,
              namedInstance="",
              userID="",
              password="")

Model is open for further operation


# Create new measured variable tables

Let's insert Variable 1 as a placeholder example.

In [22]:
s3s.GetPropertiesofElementType(ElementType=s3s.ObjectTypes.MeasuredVariableTable)

['Name',
 'Beschreibung',
 'Zeitoption',
 'Intpol',
 'Idreferenz',
 'Pk',
 'InVariant']

In [23]:
measured_variable_table_1 = s3s.InsertElement(ElementType=s3s.ObjectTypes.MeasuredVariableTable, IdRef="1")

Element inserted successfully into the model with Tk: 5563191985347834911


In [24]:
s3s.SetValue(Tk=measured_variable_table_1, propertyName="Name", Value="Variable 1")

Value is set


In [25]:
s3s.SetValue(Tk=measured_variable_table_1, propertyName="Zeitoption", Value="0") # We do not want special time reference, see general SIR 3S documentation for details

Value is set


In [26]:
s3s.SetValue(Tk=measured_variable_table_1, propertyName="Intpol", Value="0") # We choose linear interpolation scheme. Use 9 for step function

Value is set


## Insert values

W is the value that is given as a function of time W = f(t)

The sample data we provide is in 'measured' in constant time intervals and the values are random between 0 and 100.

In [27]:
# Create time vector: 0 to 600 seconds, step 20
time_seconds = np.arange(0, 601, 20)

# Create variable values between 0 and 100
np.random.seed(42)
werte = np.random.randint(0, 101, size=len(time_seconds))

YEAR = 2026
DATE = datetime(YEAR, 1, 1)  # constant date for all entries

rows = []

for t, w in zip(time_seconds, werte):
    timestamp = DATE + timedelta(seconds=int(t))
    tag = timestamp.strftime("%Y-%m-%d")
    uhrzeit = timestamp.strftime("%H:%M:%S.%f")

    rows.append({
        "TAG": tag,
        "UHRZEIT": uhrzeit,
        "WERT": w
    })

# Build DataFrame
df = pd.DataFrame(rows, columns=["TAG", "UHRZEIT", "WERT"])

In [28]:
df.head(3)

,TAG,UHRZEIT,WERT
0,2026-01-01,00:00:00.000000,51
1,2026-01-01,00:00:20.000000,92
2,2026-01-01,00:00:40.000000,14


We can use the [insert_dataframe_into_time_table()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.mantle.html#sir3stoolkit.mantle.advanced_operations.SIR3S_Model_Advanced_Operations.insert_dataframe_into_time_table) method to insert the dataframe into the model as a measured variable table.

In [29]:
s3s.insert_dataframe_into_time_table(time_table_tk=measured_variable_table_1, dataframe=df)

[2026-02-20 13:01:44,442] INFO in sir3stoolkit.mantle.dataframes: [insert dataframe into time table] Validating input data ...
[2026-02-20 13:01:44,603] INFO in sir3stoolkit.mantle.dataframes: [insert dataframe into time table] Successfully validated input data.
[2026-02-20 13:01:44,624] INFO in sir3stoolkit.mantle.dataframes: [insert dataframe into time table] Inserting value pairs ...
[2026-02-20 13:01:44,824] INFO in sir3stoolkit.mantle.dataframes: [insert dataframe into time table] Successfully inserted value pairs


Row is added to the table with Tk: 4902560581041532122
Value is set
Value is set
Row is added to the table with Tk: 5687528668222108232
Value is set
Value is set
Row is added to the table with Tk: 5416683886500773324
Value is set
Value is set
Row is added to the table with Tk: 5100816290248245811
Value is set
Value is set
Row is added to the table with Tk: 5179091624772815830
Value is set
Value is set
Row is added to the table with Tk: 4766571480628697565
Value is set
Value is set
Row is added to the table with Tk: 4959082955659840481
Value is set
Value is set
Row is added to the table with Tk: 5507931387733573574
Value is set
Value is set
Row is added to the table with Tk: 5163642874908320232
Value is set
Value is set
Row is added to the table with Tk: 5510338858160696693
Value is set
Value is set
Row is added to the table with Tk: 5278679969004524064
Value is set
Value is set
Row is added to the table with Tk: 5229431687612801046
Value is set
Value is set
Row is added to the table wi

# View measured value tables

In [30]:
def view_non_value_data_of_measured_variable_table(measured_variable_table_tk):
    name = s3s.GetValue(Tk=measured_variable_table_tk, propertyName="Name")[0]
    intpol = s3s.GetValue(Tk=measured_variable_table_tk, propertyName="Intpol")[0]
    if intpol == "0":
        intpol_clear = "linear"
    elif intpol == "9":
        intpol_clear = "step function"
    else:
        intpol_clear = "invalid intpol"
    print(f"Tk: {measured_variable_table_tk}, Name: {name}, Interpolation scheme: {intpol_clear}")

We can use the [get_dataframes_from_time_table_type()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.mantle.html#sir3stoolkit.mantle.advanced_operations.SIR3S_Model_Advanced_Operations.get_dataframes_from_time_table_type) method to obtain a df containing all dfs of a certain time table type (eg. MeasuredVariableTable), a dict with all individual dfs and their tk keys.

In [31]:
df_h, table_dfs, table_tks = s3s.get_dataframes_from_time_table_type("MeasuredVariableTable")

[2026-02-20 13:01:44,896] INFO in sir3stoolkit.mantle.dataframes: [get dataframes from time table type] ...
[2026-02-20 13:01:44,912] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Validating input data (time_table_tk=4690496339163833971)...
[2026-02-20 13:01:44,919] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Successfully validated input data.
[2026-02-20 13:01:44,924] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Building dataframe ...
[2026-02-20 13:01:45,024] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Successfully built dataframe.
[2026-02-20 13:01:45,027] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Validating input data (time_table_tk=4775965132419647289)...
[2026-02-20 13:01:45,034] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Successfully validated input data.
[2026-02-20 13:01:45,036] INFO in sir3stoolkit.mantle.dataf

## 0

In [32]:
table_dfs[table_tks[0]].head(3)

,Variable 2
Zeit [s],
0.0,0.0
18.0,1.0
22.0,0.0


In [33]:
view_non_value_data_of_measured_variable_table(table_tks[0])

Tk: 4690496339163833971, Name: Variable 2, Interpolation scheme: step function


## 1

In [34]:
table_dfs[table_tks[1]].head(3)

,Variable 2
Zeit [s],
0.0,99999.0
1.0,99999.0


As can be seen this is some invalid duplicate, that can be deleted.

In [35]:
table_tk_to_delete = table_tks[1]

## 2

In [36]:
table_dfs[table_tks[2]].head(3)

,Variable 3
Zeit [s],
0.0,1.0
20.0,35.0
40.0,57.0


In [37]:
view_non_value_data_of_measured_variable_table(table_tks[2])

Tk: 5721839716933060230, Name: Variable 3, Interpolation scheme: linear


## 3

In [38]:
table_dfs[table_tks[3]].head(3)

,Variable 1
Zeit [s],
0.0,51.0
20.0,92.0
40.0,14.0


In [39]:
view_non_value_data_of_measured_variable_table(table_tks[3])

Tk: 5563191985347834911, Name: Variable 1, Interpolation scheme: linear


This is the table we created earlier, as can be seen, it worked out fine.

# Delete measured variable table

We can delete the faulty duplicate measured variable table as any other element. The rows depending on it will be deleted as well.

In [40]:
s3s.DeleteElement(Tk=str(table_tk_to_delete))

Element deleted successfully


In [41]:
df_h, table_dfs, table_tks = s3s.get_dataframes_from_time_table_type("MeasuredVariableTable")

[2026-02-20 13:01:45,921] INFO in sir3stoolkit.mantle.dataframes: [get dataframes from time table type] ...
[2026-02-20 13:01:45,930] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Validating input data (time_table_tk=4690496339163833971)...
[2026-02-20 13:01:45,936] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Successfully validated input data.
[2026-02-20 13:01:45,938] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Building dataframe ...
[2026-02-20 13:01:45,982] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Successfully built dataframe.
[2026-02-20 13:01:45,986] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Validating input data (time_table_tk=5721839716933060230)...
[2026-02-20 13:01:45,994] INFO in sir3stoolkit.mantle.dataframes: [get dataframe from time table] Successfully validated input data.
[2026-02-20 13:01:45,997] INFO in sir3stoolkit.mantle.dataf

In [42]:
len(table_dfs)

3

# Joint view

In [43]:
df_h.head(10) 

,Variable 2,Variable 3,Variable 1
Zeit [s],,,
0.0,0.0,1.0,51.0
18.0,1.0,NaN,NaN
20.0,NaN,35.0,92.0
22.0,0.0,NaN,NaN
39.0,1.0,NaN,NaN
40.0,NaN,57.0,14.0
52.0,0.0,NaN,NaN
60.0,NaN,40.0,71.0
62.0,1.0,NaN,NaN


In [44]:
s3s.CloseModel(False)

True